## Subtypes motor task

In [11]:
import os, glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nibabel as nib

In [12]:
# load good stuff
%matplotlib inline
%load_ext rpy2.ipython
%load_ext oct2py.ipython
#sns.set(color_codes=True)

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython
The oct2py.ipython extension is already loaded. To reload it, use:
  %reload_ext oct2py.ipython


In [13]:
import warnings
warnings.filterwarnings('ignore')

In [14]:
# Add niak hcp and psomm to oactve path
%octave addpath(genpath('~/git/Misc'));
%octave build_path hcp niak psom

Adding library hcp to the search path.

Adding library niak to the search path.

Adding library psom to the search path.

In [15]:
#custom function for key sorting element
import re

def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    '''
    return [ atoi(c) for c in re.split('(\d+)', text) ]

In [16]:
#  Set path
#path_root = '/home/yassinebha/Drive/HCP/subtypes_scores/26-10-2016/subtype_5_spm_LANGUAGE_03-Oct-2017/';
path_root = '/home/yassinebha/data/data_disk/Drive/HCP2/subtypes_scores/26-10-2016/';
path_subtype = os.path.join(path_root,'subtype_5_spm_MOTOR_18-Oct-2017/');
path_association =  os.path.join(path_subtype,'associations/');
path_networks =  os.path.join(path_subtype,'networks/');
nb_sbt = natural_keys(path_subtype)[9]
nb_sbt

5

In [17]:
# List trials
list_trial  = [f for f in os.listdir(path_networks)]
list_trial 

['rf', 'lh', 't', 'lf', 'rh']

In [18]:
list_subtype = ['sub{}'.format(ii) for ii in range(1,nb_sbt+1)]
list_subtype

['sub1', 'sub2', 'sub3', 'sub4', 'sub5']

In [19]:
mask_file  =  os.path.join(path_root,'templates/mask.nii.gz')
mask_data = nib.load(mask_file).get_data()
mask = mask_data != 0
header_mask = nib.load(mask_file).get_header()
affine_mask = nib.load(mask_file).get_affine()
print('Mask NVoxels = {}'.format(np.sum(mask)))

Mask NVoxels = 76611


In [20]:
for idx,trial_name in enumerate(list_trial):
    ## Copy spec template
    command = ' '.join(['scp', os.path.join(path_root,'templates/template_spec.spec'),
                        os.path.join(path_networks,trial_name,'{}_spec.spec'.format(trial_name))])
    os.system(command)
    # Load subtypes volume
    file_subtype =glob.glob(os.path.join(path_networks,trial_name)+'/mean_subtype.*')
    file_subtype =''.join(file_subtype)
    tmp = nib.load(file_subtype).get_data()
    tmp = np.swapaxes(np.swapaxes(tmp, 0, 3), 1, 2)
    for subt_id in range(tmp.shape[3]):
        subt_net = tmp[..., subt_id]
        subt_img = nib.Nifti1Image(subt_net, affine=affine_mask,header=header_mask)
        subt_img_file = os.path.join(path_networks,trial_name,'sub{}_mean_subtype.nii.gz'.format(subt_id+1))
        subt_img.to_filename(subt_img_file)
        
        # volume to surface mapping
        file_in = subt_img_file
        list_templates =['lh','rh','cerebellum']
        for template in list_templates:
            template_file = os.path.join(path_root,'templates/{}.surf.gii'.format(template))
            file_out =  os.path.join(path_networks,trial_name,'sub{}_mean_subtype_{}.surf.gii'.format(subt_id+1,
                                                                                                      template))
            command = ' '.join(['wb_command -volume-to-surface-mapping',
                                file_in , template_file , file_out ,'-enclosing'])
            os.system(command)
            # Fill spec file
            with open(os.path.join(path_networks,trial_name,'{}_spec.spec'.format(trial_name)),"a") as f:
                if template == 'cerebellum':
                    print('   <DataFile Structure="Cerebellum"\n\
             DataFileType="METRIC"\n\
             Selected="true">\n\
     {}\n\
   </DataFile>'.format(file_out), file=f)
                elif template == 'rh':
                    print('   <DataFile Structure="CortexRight"\n\
             DataFileType="METRIC"\n\
             Selected="true">\n\
     {}\n\
   </DataFile>'.format(file_out), file=f)
                elif template == 'lh':
                    print('   <DataFile Structure="CortexLeft"\n\
             DataFileType="METRIC"\n\
             Selected="true">\n\
     {}\n\
   </DataFile>'.format(file_out), file=f)
                f.close
    with open(os.path.join(path_networks,trial_name,'{}_spec.spec'.format(trial_name)),"a") as f:
        print('</CaretSpecFile>',file=f)
        f.close

In [13]:
# %%octave -i path_root,path_subtype,path_networks,list_trial
# #split subtype to individual volume
# for nn = 1:size(list_trial)
#     trial_name = list_trial{nn};
#     subtype_folder = [path_networks filesep trial_name ];
#     file_subtype = dir([subtype_folder filesep 'mean_subtype.*']).name;
#     file_in = [subtype_folder filesep file_subtype];
#     [hdr,vol] = niak_read_vol(file_in);
#     for ii  = 1:size(vol)(4)
#         file_n = sprintf('sub%i_mean_subtype.nii.gz',ii);
#         hdr.file_name = [subtype_folder filesep file_n];
#         niak_write_vol(hdr,vol(:,:,:,1));
#         cd ([path_root filesep 'templates/'])
#         system(['sh ' path_root filesep 'templates/volume_to_surface_mapping.sh ' hdr.file_name]);
#     end
# end